In [1]:
# Install pyspark
!pip install pyspark
# Import SparkSession
from pyspark.sql import SparkSession

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [4]:
# Import a Spark function from library
from pyspark.sql.functions import col

## 1. Import Datasets

In [5]:
# Load data from csv to a dataframe. 
df_nflgames = spark.read.csv('nflgames.csv', header=True)
df_nflgames.show(5)

+------+----+----------+----------+---------+---------+---------+---------+----------+----+----+
|GameID|Week|HomeTeamID|AwayTeamID|HomeScore|AwayScore|DayOfWeek|TimeOfDay| FieldType|Temp|Wind|
+------+----+----------+----------+---------+---------+---------+---------+----------+----+----+
|     1|   1|         1|        29|       16|       23|      Sun|      Day| sportturf|  74|   8|
|     2|   2|         1|        28|       26|        6|      Thu|    Night| sportturf|  82|   6|
|     3|   3|        27|         1|       21|       23|      Sun|      Day|    grass |  71|  23|
|     4|   4|         1|        15|       38|       10|      Sun|      Day| sportturf|  78|   4|
|     5|   5|         7|         1|       20|       13|      Sun|      Day|fieldturf |   0|   0|
+------+----+----------+----------+---------+---------+---------+---------+----------+----+----+
only showing top 5 rows



In [6]:
df_nfl_players = spark.read.csv('nflplayers.csv', header=True)
df_nfl_players.show(5)

+--------+-----------------+----------+------+
|PlayerID|       PlayerName|PositionID|TeamID|
+--------+-----------------+----------+------+
|       1|      Kamar Aiken|         3|     1|
|       2|     Marlon Brown|         3|     1|
|       3|    Jeremy Butler|         3|     1|
|       4|Michael Campanaro|         3|     1|
|       5|     Owen Daniels|         4|     1|
+--------+-----------------+----------+------+
only showing top 5 rows



In [7]:
df_nfl_positions = spark.read.csv('nflpositions.csv', header=True)
df_nfl_positions.show(5)

+----------+------------+
|PositionID|PositionName|
+----------+------------+
|         1|          QB|
|         2|          RB|
|         3|          WR|
|         4|          TE|
|         5|           K|
+----------+------------+



In [8]:
df_nfl_teams = spark.read.csv('nflteams.csv', header=True)
df_nfl_teams.show(5)

+------+-------------------+------------+--------+
|TeamID|           TeamName|TeamCapsAbrv|TeamAbrv|
+------+-------------------+------------+--------+
|     1|   Baltimore Ravens|         RAV|     rav|
|     2|     Denver Broncos|         DEN|     den|
|     3|    Oakland Raiders|         RAI|     rai|
|     4|Philadelphia Eagles|         PHI|     phi|
|     5|     Dallas Cowboys|         DAL|     dal|
+------+-------------------+------------+--------+
only showing top 5 rows



## 2.	Find the team name that has the highest average HomeScore as the home team.

To find the team name, we need to first find the team ID corresponding to the highest average Homescore from the nflgames table. Thus, groupby average score according to the hometeam

In [9]:
# lets first calculate and visualise average homescore 

df_avg_homescore = df_nflgames.groupBy("HomeTeamID").agg({"HomeScore":"avg"})
df_avg_homescore.show()

+----------+------------------+
|HomeTeamID|    avg(HomeScore)|
+----------+------------------+
|         7|29.571428571428573|
|        15|18.333333333333332|
|        11|35.666666666666664|
|        29|26.166666666666668|
|         3|18.333333333333332|
|        30|              18.6|
|         8|              19.6|
|        28|              35.0|
|        22|              27.0|
|        16|16.333333333333332|
|         5|21.428571428571427|
|        31|40.833333333333336|
|        18|              26.0|
|        27|21.666666666666668|
|        17|23.333333333333332|
|        26|18.285714285714285|
|         6|              17.5|
|        19|15.166666666666666|
|        23|28.666666666666668|
|        25|              26.2|
+----------+------------------+
only showing top 20 rows



In [10]:
 # sorting in non-ascending order to extract hometeam with highest averag score
 df_avg_homescore.sort('avg(HomeScore)', ascending = False).show()

+----------+------------------+
|HomeTeamID|    avg(HomeScore)|
+----------+------------------+
|        31|40.833333333333336|
|         4|36.666666666666664|
|        11|35.666666666666664|
|         2|35.333333333333336|
|        28|              35.0|
|        20|              30.0|
|         7|29.571428571428573|
|        23|28.666666666666668|
|         1|27.166666666666668|
|        22|              27.0|
|        25|              26.2|
|        29|26.166666666666668|
|        18|              26.0|
|         9|25.666666666666668|
|        10|24.833333333333332|
|        13|              24.5|
|        32|24.333333333333332|
|        17|23.333333333333332|
|        14|22.666666666666668|
|        24|22.142857142857142|
+----------+------------------+
only showing top 20 rows



We can see TeamID 31 has the highest average homescore. We can use this ID to find out the name from the teams table

In [11]:
df_nfl_teams.filter(df_nfl_teams.TeamID==31).show()

+------+-----------------+------------+--------+
|TeamID|         TeamName|TeamCapsAbrv|TeamAbrv|
+------+-----------------+------------+--------+
|    31|Green Bay Packers|         GNB|     gnb|
+------+-----------------+------------+--------+



#### **ANS: Green Bay Packers has the highest average homescore**

## 3.	Which FieldType has the highest total scores (HomeScore + AwayScore) on average (averaging over the games in this FieldType)? Which has the lowest?

let us first create a total score column(HomeScore + AwayScore)

In [12]:
df_nflgames = df_nflgames.withColumn('total_score', df_nflgames.HomeScore + df_nflgames.AwayScore)

In [13]:
df_nflgames.show()

+------+----+----------+----------+---------+---------+---------+---------+----------+----+----+-----------+
|GameID|Week|HomeTeamID|AwayTeamID|HomeScore|AwayScore|DayOfWeek|TimeOfDay| FieldType|Temp|Wind|total_score|
+------+----+----------+----------+---------+---------+---------+---------+----------+----+----+-----------+
|     1|   1|         1|        29|       16|       23|      Sun|      Day| sportturf|  74|   8|       39.0|
|     2|   2|         1|        28|       26|        6|      Thu|    Night| sportturf|  82|   6|       32.0|
|     3|   3|        27|         1|       21|       23|      Sun|      Day|    grass |  71|  23|       44.0|
|     4|   4|         1|        15|       38|       10|      Sun|      Day| sportturf|  78|   4|       48.0|
|     5|   5|         7|         1|       20|       13|      Sun|      Day|fieldturf |   0|   0|       33.0|
|     6|   6|        19|         1|       17|       48|      Sun|      Day|    grass |  84|   6|       65.0|
|     7|   7|      

Now groupby over the fieldtype columns to find the average over all games in the specific fieldtypes

In [14]:
df_avg_FieldType = df_nflgames.groupBy("FieldType").agg({"total_score":"avg"})
df_avg_FieldType.show()

+----------+------------------+
| FieldType|  avg(total_score)|
+----------+------------------+
|    a_turf|38.166666666666664|
|    grass |46.515151515151516|
| fieldturf|              45.0|
|     grass|34.833333333333336|
|matrixturf|44.857142857142854|
| astroplay|              48.0|
|fieldturf |             47.22|
| sportturf|46.916666666666664|
+----------+------------------+



In [15]:
 # We can see astroplay has the highest total score on average(48) but lets make to more clear by sorting in non-ascending order
 df_avg_FieldType.sort('avg(total_score)', ascending = False).show()

+----------+------------------+
| FieldType|  avg(total_score)|
+----------+------------------+
| astroplay|              48.0|
|fieldturf |             47.22|
| sportturf|46.916666666666664|
|    grass |46.515151515151516|
| fieldturf|              45.0|
|matrixturf|44.857142857142854|
|    a_turf|38.166666666666664|
|     grass|34.833333333333336|
+----------+------------------+



**ANS: Astroplay has the highest total scores on average(48.0). Grass has the lowest(34.8)**

## 4. On average, how many players does a team have in each position?

Lets count positions from the players tables to find out the numer of players in each position per team. We would have to group by both the teams and the PositionID

In [16]:
df_avg_PositionID = df_nfl_players.groupBy("TeamID",'PositionID').agg({"PositionID":"count"})
df_avg_PositionID.show(truncate=False)

+------+----------+-----------------+
|TeamID|PositionID|count(PositionID)|
+------+----------+-----------------+
|31    |3         |7                |
|9     |4         |4                |
|10    |5         |1                |
|31    |1         |3                |
|22    |2         |4                |
|29    |5         |1                |
|21    |3         |5                |
|26    |1         |3                |
|9     |5         |1                |
|8     |2         |3                |
|7     |1         |2                |
|16    |2         |4                |
|4     |1         |4                |
|4     |2         |3                |
|5     |4         |4                |
|15    |2         |4                |
|25    |5         |1                |
|23    |5         |1                |
|1     |4         |3                |
|5     |1         |3                |
+------+----------+-----------------+
only showing top 20 rows



Now average over the players at each position per team

In [17]:
df_avg = df_avg_PositionID.groupBy('PositionID').agg({"count(PositionID)":"avg"})
df_avg.show()

+----------+----------------------+
|PositionID|avg(count(PositionID))|
+----------+----------------------+
|         3|               6.78125|
|         5|     1.096774193548387|
|         1|               2.71875|
|         4|                 3.875|
|         2|               4.53125|
+----------+----------------------+



OPTIONAL: number of players per position in decimal doesn't make sense; thus, we can round off figures. Also, lets rename col names

In [18]:
df_avg = df_avg.withColumnRenamed("avg(count(PositionID))","players per position per team(on average)")
from pyspark.sql.functions import round, col
df_avg = df_avg.select("*", round(col('players per position per team(on average)'))).drop('players per position per team(on average)')
df_avg = df_avg.withColumnRenamed("round(players per position per team(on average), 0)","players per position(rounded)")
df_avg.show()

+----------+-----------------------------+
|PositionID|players per position(rounded)|
+----------+-----------------------------+
|         3|                          7.0|
|         5|                          1.0|
|         1|                          3.0|
|         4|                          4.0|
|         2|                          5.0|
+----------+-----------------------------+



we can join the above table with the nfl_position table to display the position names along with the Ids

In [19]:
df_avg.join(df_nfl_positions,df_avg.PositionID ==  df_nfl_positions.PositionID,"inner")\
.select(df_avg.PositionID,'PositionName','players per position(rounded)').show()

+----------+------------+-----------------------------+
|PositionID|PositionName|players per position(rounded)|
+----------+------------+-----------------------------+
|         3|          WR|                          7.0|
|         5|           K|                          1.0|
|         1|          QB|                          3.0|
|         4|          TE|                          4.0|
|         2|          RB|                          5.0|
+----------+------------+-----------------------------+



**ANS: Above table shows the number of players a team has(on average) at each position. The numbers are rounded off. The orignal table can also be seen above.**